# Task 13 - BB84 Quantenprotokoll
Felix Kleinsteuber

Matrikelnummer: 185 709

In [2]:
import random
import numpy as np

## 1. Qubit
Wir unterscheiden zwischen zwei Basen: Der Standardbasis $\left| 0 \right\rangle, \left| 1 \right\rangle$ und der Hadamar-Basis $\left| + \right\rangle, \left| - \right\rangle$. Wird in der falschen Basis gemessen, ist das Ergebnis rein zufällig ($p(0) = p(1) = 0.5$). Wird in der korrekten Basis gemessen, ist das Ergebnis determiniert.

In [1]:
from enum import Enum
class QBasis(Enum):
    STANDARD = 0
    HADAMAR = 1

In [7]:

class Qubit:
    def __init__(self, value: int, basis: QBasis):
        assert value in [0, 1]
        self.value = value
        self.basis = basis
        self.destroyed = False
    
    def measure(self, basis: QBasis) -> int:
        if self.destroyed:
            # Wir stellen sicher, dass das Qubit nur einmal gemessen werden kann
            return -1 # Quantenzustand zerstört
        self.destroyed = True
        if basis == self.basis:
            # Basis stimmt überein: Ergebnis terminiert
            return self.value
        else:
            # Basis stimmt nicht überein: Ergebnis zufällig
            return random.getrandbits(1)

# Test: mit gleicher Basis messen
q1 = Qubit(0, QBasis.STANDARD)
assert q1.measure(QBasis.STANDARD) == 0
assert q1.measure(QBasis.STANDARD) == -1
# Test: mit unterschiedlicher Basis messen
vals = [0, 0]
for i in range(20):
    q = Qubit(0, QBasis.STANDARD)
    m = q.measure(QBasis.HADAMAR)
    assert m in [0, 1]
    vals[m] += 1
print(f"measured {vals[0]}x0, {vals[1]}x1")
assert vals[0] > 0 and vals[1] > 0
        

measured 7x0, 13x1


## 2. BB84 Protokoll

In [6]:
# Zufallszahlen erzeugen mit Quantenrechnern
def get_randbit() -> int:
    q = Qubit(0, QBasis.STANDARD)
    # wenn mit der anderen Basis gemessen wird, ist das Ergebnis zu 50% 0 und zu 50% 1
    m = q.measure(QBasis.HADAMAR)
    assert m in [0, 1]
    return m

In [19]:
class Alice:
    def __init__(self):
        self.a = []
        self.key_validated = False
    
    def step1_send_qbits(self, n: int) -> list:
        self.a = [get_randbit() for i in range(n)]
        self.a_ = [get_randbit() for i in range(n)]
        # Codiere a_i zu Quantenbits, wobei a_i' über die Basis bestimmt
        qa = [Qubit(a_i, QBasis.STANDARD if a_i_ == 0 else QBasis.HADAMAR) for a_i, a_i_ in zip(self.a, self.a_)]
        # Verschicke diese Quantenbits an Bob
        return qa
    
    def step3_compare_and_delete(self, b_: list) -> list:
        assert len(self.a_) == len(b_)
        keep_indices = np.array(self.a_) == np.array(b_)
        # Lösche alle a mit a' != b'
        self.a = np.array(self.a)[keep_indices]
        # Sende a' an Bob zum Vergleich
        return self.a_
    
    def step5_validate(self, cmp_ind: list, cmp_b: list):
        self.key_validated = np.all(self.a[cmp_ind] == cmp_b)
        if not self.key_validated:
            print(f"Validierung fehlgeschlagen! (Bob zu Alice)")
            return [0], [-1]
        else:
            # Wähle zufällige bits aus b zum Vergleichen
            cmp_ind = [random.randrange(0, len(self.a)) for i in range(len(self.a) // 6)]
            print(f"Alice: Vergleiche Bits {cmp_ind}...")
            # Sende an Bob zum Vergleichen
            return cmp_ind, self.a[cmp_ind]

    
    def get_key(self):
        return "".join([str(bit) for bit in self.a])


class Bob:
    def __init__(self):
        self.b = []
        self.key_validated = False

    def step2_measure_qbits(self, n: int, qa: list) -> list:
        assert len(qa) == n
        self.b_ = [get_randbit() for i in range(n)]
        # Miss a_i und speichere sie als b_i
        self.b = [q.measure(QBasis.STANDARD if b_i_ == 0 else QBasis.HADAMAR) for q, b_i_ in zip(qa, self.b_)]
        # Sende b' an Alice zum Vergleich
        return self.b_
    
    def step4_compare_and_delete(self, a_: list):
        assert len(a_) == len(self.b_)
        keep_indices = np.array(a_) == np.array(self.b_)
        # Lösche alle b mit a' != b'
        self.b = np.array(self.b)[keep_indices]
        # Wähle zufällige bits aus b zum Vergleichen
        cmp_ind = [random.randrange(0, len(self.b)) for i in range(len(self.b) // 6)]
        print(f"Bob: Vergleiche Bits {cmp_ind}...")
        # Sende an Alice zum Vergleichen
        return cmp_ind, self.b[cmp_ind]
    
    def step6_validate(self, cmp_ind: list, cmp_a: list):
        self.key_validated = np.all(self.b[cmp_ind] == cmp_a)
        if not self.key_validated:
            print(f"Validierung fehlgeschlagen! (Alice zu Bob)")
    
    def get_key(self):
        return "".join([str(bit) for bit in self.b])
    




### Test 1: Alles läuft glatt

In [20]:
alice = Alice()
bob = Bob()
n = 128 # 128-bit Schlüssel
qa = alice.step1_send_qbits(n)
b_ = bob.step2_measure_qbits(n, qa)
a_ = alice.step3_compare_and_delete(b_)
cmp_ind, cmp_b = bob.step4_compare_and_delete(a_)
cmp_ind, cmp_a = alice.step5_validate(cmp_ind, cmp_b)
bob.step6_validate(cmp_ind, cmp_a)
print(f"Alice: \tSchlüssel: {alice.get_key()} Validiert: {alice.key_validated}")
print(f"Bob: \tSchlüssel: {bob.get_key()} Validiert: {bob.key_validated}")

Bob: Vergleiche Bits [59, 3, 16, 4, 42, 48, 63, 61, 21, 1]...
Alice: Vergleiche Bits [7, 15, 58, 56, 2, 7, 10, 6, 51, 13]...
Alice: 	Schlüssel: 10000001100100110010100101101101010111100111111111111101101000100 Validiert: True
Bob: 	Schlüssel: 10000001100100110010100101101101010111100111111111111101101000100 Validiert: True


### Test 2: Manipulation des Quantenkanals
Eve misst jedes zweite Bit im Quantenkanal in einer zufälligen Basis und codiert den gemessenen Wert wieder in der gleichen Basis.

In [22]:
class Eve():
    def __init__(self):
        self.sneaked_bits = []
    def sneak(self, qa: list):
        # Eve misst jedes zweite Bit in qa in einer zufälligen Basis, um so durchschnittlich die Hälfte des halben Schlüssels zu erhalten
        for i in range(0, len(qa)):
            if i % 2 == 0:
                basis_bit = get_randbit()
                basis = QBasis.STANDARD if basis_bit == 0 else QBasis.HADAMAR
                m = qa[i].measure(basis)
                self.sneaked_bits.append(m)
                # da Eve das gemessene Bit zerstört hat, muss sie ein neues präparieren
                qa[i] = Qubit(m, basis)
            else:
                # unbekanntes Bit (wird nicht gemessen)
                self.sneaked_bits.append("x")
        return qa
    
    def get_key(self):
        return "".join([str(bit) for bit in self.sneaked_bits])


In [23]:
alice = Alice()
bob = Bob()
eve = Eve()
n = 128 # 128-bit Schlüssel
qa = alice.step1_send_qbits(n)
qa = eve.sneak(qa)
b_ = bob.step2_measure_qbits(n, qa)
a_ = alice.step3_compare_and_delete(b_)
cmp_ind, cmp_b = bob.step4_compare_and_delete(a_)
cmp_ind, cmp_a = alice.step5_validate(cmp_ind, cmp_b)
bob.step6_validate(cmp_ind, cmp_a)
print(f"Alice: \tSchlüssel: {alice.get_key()} Validiert: {alice.key_validated}")
print(f"Bob: \tSchlüssel: {bob.get_key()} Validiert: {bob.key_validated}")
print(f"Eve: \tSchlüssel: {eve.get_key()}")

Bob: Vergleiche Bits [29, 9, 31, 49, 24, 54, 60, 44, 25, 23]...
Validierung fehlgeschlagen! (Bob zu Alice)
Validierung fehlgeschlagen! (Alice zu Bob)
Alice: 	Schlüssel: 00111010011100110100101111001100010110100101011111000010000001 Validiert: False
Bob: 	Schlüssel: 00111010011000010100000110001100010111100101001111010000100101 Validiert: False
Eve: 	Schlüssel: 0x0x0x1x1x1x0x0x0x0x1x1x0x0x0x1x1x1x1x1x0x0x1x1x1x1x1x1x0x0x0x0x1x0x0x0x0x1x1x0x0x1x0x0x0x0x1x1x0x0x1x1x0x0x0x1x0x0x1x0x0x0x0x1x


**Erfolgreich:** Die Validierung schlägt fehl.